In [ ]:
import veloxchem as vlx
from pandas import *
from rdkit import Chem

In [ ]:
data = read_csv("nucleophilicity_with_names.csv")
smiles_list = data['smiles'].tolist()

In [ ]:
mol = vlx.Molecule.read_smiles(smiles_list[1])
mol.show()
rd_mol = Chem.MolFromSmiles(smiles_list[1])
charge = Chem.GetFormalCharge(rd_mol)

In [ ]:
results = []
for i in range(len(smiles_list)):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.level_shifting = 0.0
    scf_drv.xcfun = "pbe"
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    scf_drv.ostream.mute()
    result = scf_drv.compute(mol, basis)
    if result == None:
        scf_drv.level_shifting = 1.0
        result = scf_drv.compute(mol, basis)
        if result == None:
            print("SCF failed")
        else:
            results.append(result['scf_energy'])
            print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1) + " with level shifting 1.0")
    else:
        results.append(result['scf_energy'])
        print(str("Energy: " + str(result['scf_energy'])) + " of structure " + str(i+1))
print("Finished")

In [ ]:
opt_geometries = []
opt_energies = []
for i in range(10):
    mol = vlx.Molecule.read_smiles(smiles_list[i])
    rd_mol = Chem.MolFromSmiles(smiles_list[i])
    charge = Chem.GetFormalCharge(rd_mol)
    mol.set_charge(charge)
    basis = vlx.MolecularBasis.read(mol, 'def2-svp')
    scf_drv = vlx.ScfRestrictedDriver()
    scf_drv.ri_coulomb = True
    scf_drv.grid_level = 2
    scf_drv.level_shifting = 1.0
    scf_drv.xcfun = "pbe"
    scf_drv.ostream.mute()
    scf_drv.dispersion = True
    scf_drv.conv_thresh = 1e-4
    result = scf_drv.compute(mol, basis)

    if result == None:
        scf_drv.level_shifting = 1.0
        result = scf_drv.compute(mol, basis)
        if result == None:
            print("SCF failed")
            continue
        
    opt_drv = vlx.OptimizationDriver(scf_drv)
    scf_drv.level_shifting = 1.0
    opt_drv.conv_energy = 1e-04
    opt_drv.conv_drms = 1e-02
    opt_drv.conv_dmax = 2e-02
    opt_drv.conv_grms = 4e-03
    opt_drv.conv_gmax = 8e-03
    opt_results = opt_drv.compute(mol, basis, result)
    opt_geometries.append(opt_results['final_geometry'])
    opt_energies.append(opt_results['opt_energies'][-1])
    print(str("Energy: " + str(opt_results['opt_energies'][-1])) + " of optimized structure " + str(i+1) + ". Optimized in " + str(len(opt_results['opt_energies'])) + " iterations.")
print("Finished")